In [1]:
!pip install opendatasets

In [2]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.utils import load_img, img_to_array
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam

In [3]:
od.download("https://www.kaggle.com/datasets/catherinehorng/dogbreedidfromcomp")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/catherinehorng/dogbreedidfromcomp


100%|██████████| 691M/691M [03:04<00:00, 3.93MB/s] 


In [5]:
labels_all= pd.read_csv("dogbreedidfromcomp\\labels.csv")

In [6]:
labels_all.head(
    
)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [7]:
print(labels_all.shape)

(10222, 2)


In [10]:
labels_all["breed"].unique()

array(['boston_bull', 'dingo', 'pekinese', 'bluetick', 'golden_retriever',
       'bedlington_terrier', 'borzoi', 'basenji', 'scottish_deerhound',
       'shetland_sheepdog', 'walker_hound', 'maltese_dog',
       'norfolk_terrier', 'african_hunting_dog',
       'wire-haired_fox_terrier', 'redbone', 'lakeland_terrier', 'boxer',
       'doberman', 'otterhound', 'standard_schnauzer',
       'irish_water_spaniel', 'black-and-tan_coonhound', 'cairn',
       'affenpinscher', 'labrador_retriever', 'ibizan_hound',
       'english_setter', 'weimaraner', 'giant_schnauzer', 'groenendael',
       'dhole', 'toy_poodle', 'border_terrier', 'tibetan_terrier',
       'norwegian_elkhound', 'shih-tzu', 'irish_terrier', 'kuvasz',
       'german_shepherd', 'greater_swiss_mountain_dog', 'basset',
       'australian_terrier', 'schipperke', 'rhodesian_ridgeback',
       'irish_setter', 'appenzeller', 'bloodhound', 'samoyed',
       'miniature_schnauzer', 'brittany_spaniel', 'kelpie', 'papillon',
       'borde

In [11]:
breed_all = labels_all["breed"]

In [12]:
breed_all = breed_all.value_counts()

In [13]:
breed_all

breed
scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
                       ... 
golden_retriever         67
brabancon_griffon        67
komondor                 67
eskimo_dog               66
briard                   66
Name: count, Length: 120, dtype: int64

In [15]:
class_names= ["scottish_deerhound","maltese_dog","afghan_hound"]
labels = labels_all[(labels_all["breed"].isin(class_names))]
labels = labels.reset_index()
labels

,index,id,breed
0,9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
1,12,00693b8bc2470375cc744a6391d397ec,maltese_dog
2,79,01e787576c003930f96c966f9c3e1d44,scottish_deerhound
3,90,022b34fd8734b39995a9f38a4f3e7b6b,maltese_dog
4,146,0379145880ad3978f9b80f0dc2c03fba,afghan_hound
...,...,...,...
354,9927,f98392bfae7d0aa35ecc4993ee2afbaf,maltese_dog
355,9972,fa5054c5187c7171c546bd6a46b8346f,maltese_dog
356,10158,fe50bac6c389d137ea01c9cfc7346ca8,scottish_deerhound
357,10200,ff7d9c08091acc3b18b869951feeb013,maltese_dog


In [16]:
X_data = np.zeros((len(labels),224,224,3),dtype="float32")
Y_data = label_binarize(labels["breed"],classes=class_names)

In [18]:
for i in tqdm(range(len(labels))):
    img= load_img("dogbreedidfromcomp//train//%s.jpg"%labels["id"][i],target_size=[224,224])
    img= img_to_array(img)
    x = np.expand_dims(img.copy(),axis=0)
    X_data[i]=x/255.0
print(X_data.shape)
print(Y_data.shape)

100%|██████████| 359/359 [00:06<00:00, 56.03it/s]

(359, 224, 224, 3)
(359, 3)


In [ ]:
model = Sequential()

model.add(Conv2D(filters = 64, 
                 kernel_size=(5,5), 
                 activation="relu",
                 input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 32,
                 kernel_size=(3,3),
                 activation="relu",
                 kernel_regularizer="l2"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 16,
                 kernel_size=(7,7),
                 activation="relu",
                 kernel_regularizer="l2"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters = 8,
                 kernel_size=(5,5),
                 activation="relu",
                 kernel_regularizer="l2"))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation="relu", kernel_regularizer="l2"))
model.add(Dense(64,activation="relu", kernel_regularizer="l2"))
model.add(Dense(len(class_names), activation="softmax"))

model.compile(loss = "categorical_crossentropy",
              optimizer = Adam(0.0001), 
              metrics = ["accuracy"])

model.summary()